In [181]:
from ipynb.fs.full.feature_engineering_final import X_Transformer_scaled
from ipynb.fs.full.feature_engineering_final import X_Transformer
from ipynb.fs.full.feature_engineering_final import DataLoader
from ipynb.fs.full.feature_engineering_final import TargetNormalizedRegressor
from lightgbm import LGBMRegressor
from ipynb.fs.full.feature_engineering_final import BlendingRegressor
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import ElasticNetCV
import xgboost as xgb
import numpy as np
import pandas as pd
from xgboost import plot_tree
from mlxtend.regressor import StackingCVRegressor
import matplotlib.pyplot as plt
%matplotlib inline
from xgboost import plot_tree
from matplotlib.pylab import rcParams
from sklearn.model_selection import KFold
from sklearn.svm import SVR

In [182]:
data_loader = DataLoader("train.csv","test.csv")

/home/victor/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:3399: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/victor/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:3429: PearsonRNearConstantInputWarning: An input array is nearly constant; the computed correlation coefficent may be inaccurate.
  warnings.warn(PearsonRNearConstantInputWarning())


In [183]:
X_train = data_loader.getX_train()
Y_train = data_loader.getY_train()
X_test = data_loader.getX_test()

In [184]:
x_transformer = X_Transformer_scaled()

In [185]:
kf = KFold(n_splits=10,shuffle = True,random_state = 0)

In [186]:
xgb = XGBRegressor(objective='reg:squarederror',eval_metric = 'rmse',learning_rate=0.01,n_estimators=4900,
                                     max_depth=3, min_child_weight=0,
                                     gamma=0, subsample=0.7,
                                     colsample_bytree=0.7,
                                     nthread=-1,
                                     scale_pos_weight=1, seed=27,
                                     reg_alpha=0.00006)
xgb_t = TargetNormalizedRegressor(xgb)

svr = SVR(kernel="rbf",gamma=0.0003,C = 14,epsilon = 0.00008)

svr_t = TargetNormalizedRegressor(svr)

ridge = RidgeCV(cv=10)

ridge_t = TargetNormalizedRegressor(ridge)

lasso = LassoCV(cv=10)
lasso_t = TargetNormalizedRegressor(lasso)

elastic_net = ElasticNetCV(max_iter=1e7, alphas=[0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007],
                                        cv=kf, l1_ratio=[0.8, 0.85, 0.9, 0.95, 0.99, 1])
elastic_net_t = TargetNormalizedRegressor(elastic_net)

gbr = GradientBoostingRegressor(n_estimators=5950, learning_rate=0.01,
                                   max_depth=4, max_features='sqrt',
                                   loss='huber', random_state =42,subsample = 0.7)

gbr_t = TargetNormalizedRegressor(gbr)


scvr = StackingCVRegressor(regressors=(xgb,svr,ridge,lasso,elastic_net,gbr),
                                meta_regressor=ridge,cv=15,
                                use_features_in_secondary=True,
                                  n_jobs = -1)
scvr_t = TargetNormalizedRegressor(scvr)

lightgbm = LGBMRegressor(objective='regression',num_leaves=4, learning_rate=0.01, 
              n_estimators=5000, max_bin=200, 
              bagging_fraction=0.75, bagging_freq=1, 
              bagging_seed=7, feature_fraction=0.2,feature_fraction_seed=7,
                                       verbose=-1)

#blrgr = TargetNormalizedRegressor(BlendingRegressor(regressors_and_weights = [(xgb,0.139342855),(svr,0.140628783),(ridge,0.143950385),(lasso,0.1430492),(elastic_net,0.144500587),(gbr,0.141916906)]))
#blrgr = BlendingRegressor(regressors_and_weights = [(svr_t,0.5)])


In [187]:
learning_pipeline = Pipeline(steps=[('preprocess', x_transformer),('learn',gbr_t)])
learning_pipeline.fit(X_train,y=Y_train)





Pipeline(memory=None,
         steps=[('preprocess',
                 Pipeline(memory=None,
                          steps=[('col_drop',
                                  ColumnDrop(columns=['Utilities', 'Street',
                                                      'PoolQC'])),
                                 ('col_transformer',
                                  ColumnTransformer(n_jobs=None,
                                                    remainder='passthrough',
                                                    sparse_threshold=0.3,
                                                    transformer_weights=None,
                                                    transformers=[('categorical_with_missing_values_const',
                                                                   PipelineWithFeatureNames(feature_names=['...
                                                                               learning_rate=0.01,
                                                    

In [188]:
#set up the parameters
#rcParams['figure.figsize'] = 100,90
#plot_tree(learning_pipeline.named_steps['learn'],num_trees=2799, rankdir='LR')#plt.show()


In [189]:
Ypredicted = learning_pipeline.predict(X_test)
#Ypredicted =   #pd.read_csv('submission_stack_elasticnet.csv').iloc[:,1]
#Ypredicted = pd.read_csv('submission_stack_ridge.csv').iloc[:,1]*0.5 + pd.read_csv('submission_stack_elasticnet.csv').iloc[:,1]*0.5

In [190]:
resultDf = pd.DataFrame()
resultDf['Id'] = data_loader.get_Test_id()
resultDf['SalePrice'] = Ypredicted

#q1 = resultDf['SalePrice'].quantile(0.0045)
#q2 = resultDf['SalePrice'].quantile(0.99)
#resultDf[resultDf['SalePrice'] <= q1]
#resultDf[resultDf['SalePrice'] <= q1]

#resultDf['SalePrice'] = resultDf['SalePrice'].apply(lambda x: x if x > q1 else x*0.9)
#resultDf['SalePrice'] = resultDf['SalePrice'].apply(lambda x: x if x < q2 else x*1.081)

#resultDf
resultDf.to_csv('submission_gbr.csv',index = False)